# Pivoted QR

Here's another look at our modified Gram-Schmidt algorithm for QR.

In [1]:
using LinearAlgebra

function mgs(A)
    m,n = size(A)
    Q = zeros(m,n)
    R = zeros(n,n)
    B = copy(A)
    for k = 1:n
        R[k,k] = norm(B[:,k])
        Q[:,k] = B[:,k]/R[k,k]
        for j = k+1:n
            R[k,j] = Q[:,k]⋅B[:,j]
        end
        B -= Q[:,k]*R[k,:]'
    end
    return Q,R
end 

mgs (generic function with 1 method)

It's apparent that something bad happens if for some $k$, $R_{kk}=0$. In terms of $A=QR$, one interpretation of this event is that the columns $a_1,\ldots,a_k$ all lie in the span of $q_1,\ldots,q_{k-1}$. This implies a defect in the dimension of the column space; i.e., the rank of $A$ is less than $n$. 

In machine arithmetic, which is inexact for most numbers, it's rare for something to be exactly zero. But the behavior is often about as bad for "near zero" as for zero itself. Let's set out to avoid this situation by replacing $R_{kk}$ with whatever is as large as possible, in context.

Again considering $A = \sum_{k=1}^n q_k r_k^T$, define $A_j=\sum_{k=j}^n q_k r_k^T$. The MGS algorithm starts off by noting 

$$Ae_1=A_1e_1 = \sum_{k=1}^n q_k (r_k^Te_1) = R_{11}q_1,$$

where to simplify the sum we applied the upper triangular structure of $R$, specifically, $R_{21}=\cdots=R_{m1}=0$. This formula is used immediately to compute $R_{11}$, then $q_1$, both from the first column $A_1e_1$.

But if $R_{11}$ is zero (or small), this creates a problem. So instead suppose that we use the column of $A$ that produces the largest possible value. That is, let 

$$j_1 = \text{argmax}_k \| A_1 e_k \|_2,$$

and then replace the triangular requirement with 

$$R_{2,j_1}=\cdots=R_{m,j_1}=0.$$

This creates $A_1 e_{j_1}=R_{1,j_1}q_{1}$, from which we get $R_{1,j_1}$ and then $q_{1}$. Furthermore, $q_{1}^TA_1=r_{1}^T$, just as before, allowing us to fill out the rest of the first row of $R$. 

For example,

In [2]:
Q = zeros(6,3)
R = zeros(3,3)
A = rand(1.:10,6,3)

6×3 Array{Float64,2}:
 3.0   3.0  10.0
 7.0   7.0   5.0
 3.0   8.0   3.0
 1.0   8.0   1.0
 3.0   4.0   6.0
 9.0  10.0   1.0

In [3]:
A1 = copy(A)
c = [ sqrt(sum(A1[i,j]^2 for i=1:6)) for j=1:3 ]

3-element Array{Float64,1}:
 12.569805089976535
 17.378147196982766
 13.114877048604   

In [4]:
j = fill(0,3)
cmax,j[1] = findmax(c)

(17.378147196982766, 2)

In [5]:
R[1,j[1]] = cmax
R

3×3 Array{Float64,2}:
 0.0  17.3781  0.0
 0.0   0.0     0.0
 0.0   0.0     0.0

In [6]:
Q[:,1] = A1[:,j[1]]/R[1,j[1]]
cols = fill(true,3)
cols[j[1]] = false
R[1,cols] = Q[:,1]'*A1[:,cols]
R

3×3 Array{Float64,2}:
 11.0484  17.3781  7.5382
  0.0      0.0     0.0   
  0.0      0.0     0.0   

By our definitions, $A_2=A_1-q_1r_1^T$.

In [7]:
A2 = A1 - Q[:,1]*R[1,:]'

6×3 Array{Float64,2}:
  1.09272    0.0           8.69868 
  2.54967   -8.88178e-16   1.96358 
 -2.08609    0.0          -0.470199
 -4.08609    0.0          -2.4702  
  0.456954   0.0           4.2649  
  2.64238    0.0          -3.33775 

Note that we have zeroed out the third column. In fact, we're not going to touch that column of $R$ any more (since doing so would wreck the quasi-triangularity we set up). Instead, we choose the next column like we did at the start.

In [8]:
c = [ sqrt(sum(A2[i,j]^2 for i=1:6)) for j=1:3 ]
cmax,j[2] = findmax(c)

(10.731984750675977, 3)

Now the game is to set $R_{k,j_2}=0$ for all $k>2$. Doing so means that 

$$A_2 e_{j_2}=\sum_{k=2}^m q_k (r_k^Te_{j_2}) =  R_{2,j_2}q_2.$$

And then also $q_2^T A_2=r_2^T$. 

In [9]:
R[2,j[2]] = cmax
Q[:,2] = A2[:,j[2]]/R[2,j[2]]
cols[j[2]] = false
R[2,cols] = Q[:,2]'*A2[:,cols]
R

3×3 Array{Float64,2}:
 11.0484   17.3781   7.5382
  1.74387   0.0     10.732 
  0.0       0.0      0.0   

So we have $A_3=A_2-q_2r_2^T$:

In [10]:
A3 = A2 - Q[:,2]*R[2,:]'

6×3 Array{Float64,2}:
 -0.32076    0.0          0.0
  2.2306    -8.88178e-16  0.0
 -2.00969    0.0          0.0
 -3.6847     0.0          0.0
 -0.236064   0.0          0.0
  3.18475    0.0          0.0

Another column zeroed out! Etc.

In [11]:
c = [ sqrt(sum(A3[i,j]^2 for i=1:6)) for j=1:3 ]
cmax,j[3] = findmax(c)

(5.735213810955839, 1)

In [12]:
R[3,j[3]] = cmax
Q[:,3] = A3[:,j[3]]/R[3,j[3]];
R

3×3 Array{Float64,2}:
 11.0484   17.3781   7.5382
  1.74387   0.0     10.732 
  5.73521   0.0      0.0   

So now we have a factorization...

In [13]:
norm(A-Q*R)

1.0295784775289034e-15

...but $R$ is no longer upper triangular. However, it's virtually so. In column $j_1$ we enforced zero below row 1, in column $j_2$ we enforced zero below row 2, and so on. So if we take the columns in order $j_1,\ldots,j_m$, the triangularity is restored.

In [14]:
R[:,j]

3×3 Array{Float64,2}:
 17.3781   7.5382  11.0484 
  0.0     10.732    1.74387
  0.0      0.0      5.73521

This reordering might be expressed using right-multiplication by a permutation matrix $P$. In fact, we might say $RP = U$ for a truly triangular $U$, or $A=QUP^{-1}$. However, it's customary to put the permutation next to $A$ and continue using $R$ to mean a truly upper triangular matrix: $AP=QR$. This is a **pivoted QR factorization**. 

I'm not going to put a complete algorithm here. In practice one doesn't use MGS for this anyway, and there are some further efficiencies I've skipped. 

The default in Julia is to get an *unpivoted* QR.

In [15]:
fact=qr(A);
fact.R

3×3 Array{Float64,2}:
 -12.5698  -15.2747   -8.11468
   0.0       8.28755   0.85075
   0.0       0.0      10.2678 

However, you can request the pivoted form.

In [16]:
fact = qr(A,Val(true));

In [17]:
fact.R

3×3 Array{Float64,2}:
 -17.3781  -7.5382  -11.0484 
   0.0     10.732     1.74387
   0.0      0.0       5.73521

This is the same as ours (up to arbitrary signs). The permutation is available, as with our vector `j`.

In [18]:
fact.p

3-element Array{Int64,1}:
 2
 3
 1